In [ ]:

# COMPAS Dataset Bias Audit with AIF360

# Install AIF360
!pip install aif360

import pandas as pd
import numpy as np
from aif360.datasets import CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.postprocessing import RejectOptionClassification
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Load COMPAS dataset
dataset_orig = CompasDataset(protected_attribute_names=['race'],
                             privileged_classes=[['Caucasian']],
                             features_to_drop=['sex', 'name', 'id'])

# Split into train and test
dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)

# Baseline Fairness Metrics
metric_orig_train = BinaryLabelDatasetMetric(dataset_orig_train, 
    unprivileged_groups=[{'race': 1}], 
    privileged_groups=[{'race': 0}])

print("Mean Difference (privileged - unprivileged):", metric_orig_train.mean_difference())
print("Disparate Impact:", metric_orig_train.disparate_impact())

# Reweighing (Mitigation)
RW = Reweighing(unprivileged_groups=[{'race': 1}], privileged_groups=[{'race': 0}])
dataset_transf_train = RW.fit_transform(dataset_orig_train)

# Train Logistic Model
scaler = StandardScaler()
X_train = scaler.fit_transform(dataset_transf_train.features)
y_train = dataset_transf_train.labels.ravel()
w_train = dataset_transf_train.instance_weights

model = LogisticRegression()
model.fit(X_train, y_train, sample_weight=w_train)

# Test on Original Test Set
X_test = scaler.transform(dataset_orig_test.features)
y_pred = model.predict(X_test)
dataset_pred = dataset_orig_test.copy()
dataset_pred.labels = y_pred

# Evaluate Bias & Accuracy
classified_metric = ClassificationMetric(dataset_orig_test, dataset_pred,
                                         unprivileged_groups=[{'race': 1}],
                                         privileged_groups=[{'race': 0}])

print("Accuracy:", accuracy_score(dataset_orig_test.labels, y_pred))
print("Disparate Impact:", classified_metric.disparate_impact())
print("Equal Opportunity Diff:", classified_metric.equal_opportunity_difference())
print("Average Odds Diff:", classified_metric.average_odds_difference())
print("False Positive Rate Diff:", classified_metric.false_positive_rate_difference())

# Visualization
labels = ['Disparate Impact', 'Equal Opportunity Diff', 'Avg Odds Diff', 'FPR Diff']
values = [classified_metric.disparate_impact(),
          classified_metric.equal_opportunity_difference(),
          classified_metric.average_odds_difference(),
          classified_metric.false_positive_rate_difference()]

plt.figure(figsize=(10, 5))
plt.bar(labels, values, color='salmon')
plt.title('Fairness Metrics After Mitigation')
plt.axhline(0, color='gray', linestyle='--')
plt.show()
